In [1]:
from PySide6.QtSql import QSqlDatabase
from mysql import connector

conn = connector.connect(    
    host='localhost',
    user='root',
    password='root')

cursor = conn.cursor()


In [5]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
for table in tables:
    print(table)
    cursor.execute(f"DESC {table[0]}")
    print(cursor.fetchall())
    print()
    

('course',)
[('course_name', 'varchar(100)', 'NO', 'PRI', None, ''), ('teacher_username', 'varchar(100)', 'NO', 'MUL', None, '')]

('course_student',)
[('student_id', 'int', 'YES', 'MUL', None, ''), ('course_name', 'varchar(100)', 'YES', 'MUL', None, '')]

('student',)
[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('gender', "enum('男','女')", 'YES', '', None, ''), ('age', 'char(3)', 'YES', '', None, ''), ('name', 'varchar(100)', 'YES', '', None, '')]

('teacher',)
[('name', 'varchar(100)', 'NO', '', None, ''), ('username', 'varchar(100)', 'NO', 'PRI', None, ''), ('password', 'varchar(100)', 'NO', '', None, '')]



In [9]:
# 该格子用于创建对应的表。包括学生表，教师表，课程表，课程学生表，考勤表

# 开始事务
conn.start_transaction()

try:
    # Create the schema
    create_schema = "CREATE SCHEMA IF NOT EXISTS attendance_system"
    cursor.execute(create_schema)

    # Switch to the new schema
    cursor.execute("USE attendance_system")

    # Create the teacher table
    create_teacher_table = """
    CREATE TABLE IF NOT EXISTS teacher (
        name VARCHAR(100) NOT NULL,
        username VARCHAR(100) NOT NULL PRIMARY KEY,
        password VARCHAR(100) NOT NULL
    )
    """
    cursor.execute(create_teacher_table)

    # Create the student table
    create_student_table = """
    CREATE TABLE IF NOT EXISTS student (
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        gender ENUM('男', '女') NULL,
        age CHAR(3) NULL,
        name VARCHAR(100) NULL
    )
    """
    cursor.execute(create_student_table)

    # Create the course table
    create_course_table = """
    CREATE TABLE IF NOT EXISTS course (
        course_name VARCHAR(100) NOT NULL PRIMARY KEY,
        teacher_username VARCHAR(100) NOT NULL,
        FOREIGN KEY (teacher_username) REFERENCES teacher(username) On DELETE CASCADE
    )
    """
    cursor.execute(create_course_table)

    # Create the course_student table
    create_course_student_table = """
    CREATE TABLE IF NOT EXISTS course_student (
        course_name VARCHAR(100) NOT NULL,
        student_id INT NOT NULL,
        PRIMARY KEY (course_name, student_id),
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE,
        FOREIGN KEY (student_id) REFERENCES student(id) ON DELETE CASCADE
    )
    """
    cursor.execute(create_course_student_table)

    # Create the attendance_info table
    create_attendance_info_table = """
    CREATE TABLE IF NOT EXISTS attendance_info (
        date DATE NOT NULL,
        stu_id INT NOT NULL,
        course_name VARCHAR(100) NOT NULL,
        is_attendance BOOLEAN NOT NULL,
        PRIMARY KEY (date, stu_id, course_name),
        FOREIGN KEY (stu_id) REFERENCES student(id) ON DELETE CASCADE,
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE
    )
    """
    cursor.execute(create_attendance_info_table)

    # Commit the changes
    conn.commit()
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"An error occurred: {e}")

In [10]:
# Retrieve the list of tables in the schema
cursor.execute("""
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'attendance_system'
""")
tables = cursor.fetchall()

# Delete the content of each table
try:
    try:
        conn.rollback()
        print("回滚了之前可能存在的事务")
    except:
        pass
    # Start transaction
    conn.start_transaction()

    # Disable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

    for table in tables:
        table_name = table[0]
        cursor.execute(f"DELETE FROM {table_name}")
        print(f"All content deleted from table: {table_name}")

    # Enable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

    # Commit transaction
    conn.commit()
except Exception as e:
    # Rollback transaction in case of error
    conn.rollback()
    print(f"An error occurred: {e}")


In [2]:
query = '''
        SHOW TABLES'''

cursor.execute(query)
result = cursor.fetchall()
result

[('new',)]

In [3]:
q = """
    SELECT name
    FROM new
    WHERE id = 1"""

cursor.execute(q)
res = cursor.fetchall()
res

[('x',)]

In [15]:
query = """
        SELECT COUNT(*)
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE table_name = 'class1';
"""
table = 'class1'
cursor.execute('SELECT COUNT(*) FROM %s' %  table)
results = cursor.fetchone()


In [14]:
print(results)

(4,)


In [ ]:
table_name = 'class1'
cursor.execute("""
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = '%s'
""" % table_name)

# 获取列名



In [19]:
table_name = 'class1'
sql =f"INSERT INTO {table_name} (name, gender, age) VALUES(%s, %s, %s)"
cursor.execute(sql, [ 'e', 'male', 18])  


In [18]:
cursor.execute('SELECT * FROM class1')
results = cursor.fetchall()
print(results)

[(1, 'a', 'female', 18), (2, 'b', 'male', 18), (3, 'c', 'male', 18), (4, 'd', 'male', 18)]


In [3]:
sql = f'SELECT id, name FROM test'
cursor.execute(sql)
res = cursor.fetchall()

In [4]:
res

[(1, 'a'), (2, 'trump')]